In [ ]:
# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])
import requests
import json

# this function takes a movie title and returns tastedive dictionary
def get_movies_from_tastedive(movie_input):
    baseurl = 'https://tastedive.com/api/similar'
    param_diction = {}
    param_diction['q'] = movie_input
    param_diction['type'] = 'movies'
    param_diction['limit'] = '5'
    result_obj = requests.get(baseurl,params=param_diction)
    result_dictionary = result_obj.json()
    return result_dictionary

# This function takes tastedive dictionary returns the list of 5 movies
def extract_movie_titles(result_dictionary):
    movies_list = [movie_dictionary['Name'] for movie_dictionary in result_dictionary['Similar']['Results']]
    return movies_list

# This function takes a movie list and returns a list of similar movies
def get_related_titles(movie_list_to_search):
    # aggregate_movie_list = [extract_movie_titles(a_movie) for a_movie in list_of_movies_to_search]
    final_movie_list = []
    for movie in movie_list_to_search:
        result_dictionary = get_movies_from_tastedive(movie)
        prelim_movie_list = extract_movie_titles(result_dictionary)
        for movie_result in prelim_movie_list:
            if movie_result not in final_movie_list:
                final_movie_list.append(movie_result)
    return final_movie_list

# This function takes a movie title and return OMDB dictionary for single movie
def get_movie_data(movie_title):
    baseurl = 'http://www.omdbapi.com/'
    search_params = {}
    search_params['t'] = movie_title
    search_params['r'] = 'json'
    request_obj = requests.get(baseurl, params = search_params)
    result_diction = request_obj.json()
    return result_diction
# This function takes OMDB dictionary for single movie
def get_movie_rating(omdb_dictionary):
    for rating_source_diction in omdb_dictionary['Ratings']:
        if rating_source_diction['Source'] == 'Rotten Tomatoes':
            return (int(rating_source_diction['Value'].strip('%')))
    return 0

def get_sorted_recommendations(movie_list_input):
    similar_movies = get_related_titles(movie_list_input)
    movie_recommendation_d = {}
    for similar_movie in similar_movies:
        omdb_dictionary = get_movie_data(similar_movie)
        rotten_tomatoe_score = get_movie_rating(omdb_dictionary)
        movie_recommendation_d[similar_movie] = rotten_tomatoe_score
    return sorted(movie_recommendation_d.keys(), reverse = True,key = lambda k: (movie_recommendation_d[k],k))  
    
print(get_sorted_recommendations(['Black Panther']))  